In [ ]:
import os
import sys
import re
import json
import pandas as pd 
import numpy as np
import seaborn as sns; sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)
    
import run_gnssrefl 
import gnssrefl_helpers

%matplotlib inline

In [ ]:
#Making sure environment variables are set
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment()
else:
     print('environment variable ORBITS path is:\n', os.environ['ORBITS'],
          '\nenvironment variable REFL_CODE path is:\n', os.environ['REFL_CODE'],
          '\nenvironment variable EXE path is:\n', os.environ['EXE'])
        
refl_code_loc = os.environ['REFL_CODE']
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
gnssrefl_helpers.download_crx2rnx()

# Ross Ice Shelf, Antarctica

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



### Data Summary

Station lorg is on the Ross Ice Shelf, Antarctica.
The site is a largely featureless ice plain with no obstructions (see photo above). 
The site was installed on November 27, 2018 and decommissioned and removed on November 15, 2019. 
It recorded only GPS frequencies during its operation. 

LORG is an example station on the [GNSS-IR web app.](https://gnss-reflections.org/api?example=lorg) 

There are no significant topographic features near the station, so it is recommended to use default values 
for the elevation mask. An azimuth mask is not required.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=lorg" width="1000" height="900"></iframe>

## Take a look at the Data

First make a SNR file.

In [ ]:
station = 'lorg'
year = 2019 
doy = 205

lat = -78.1836
long = 170.0336
height = -7.722

In [ ]:
# Can view the parameters here
#run_gnssrefl.rinex2snr?

In [ ]:
run_gnssrefl.rinex2snr(station,year,doy, translator='hybrid')



Use **quickLook** to produce a periodogram similar to the one in the web app [(For details on quickLook output)](../../docs/quickLook_desc.md). quickLook is set to use the L1 frequency by default:

In [ ]:
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy)
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook: {station}', size=16)

quadrants = ['NW', 'NE', 'SW', 'SE']

for i, ax in enumerate(axes.flat):
    for fail_satellite in values[f'f{quadrants[i]}']:
        g = sns.lineplot(x=values[f'f{quadrants[i]}'][fail_satellite][0], y=values[f'f{quadrants[i]}'][fail_satellite][1], ax=ax, color='lightgrey')
    for satellite in values[quadrants[i]]:
        g = sns.lineplot(x=values[quadrants[i]][satellite][0], y=values[quadrants[i]][satellite][1], ax=ax)
    g.set_ylabel('volts/volts')
    g.set_xlabel('reflector height (m)')

plt.tight_layout()
plt.show()

Compare the periodograms for other frequencies: L2C and L5. They should be similar to the L1 periodogram, except that there
will be fewer satellite traces because only GPS satellites launched after 2005
broadcast L2C and only satellites after 2010 broadcast L5.
The northwest qudarant is the noisiest and one could certainly try to improve the results by restricting some azimuths there.
This is for L2C:

In [ ]:
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, f=20)
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook: {station}', size=16)

quadrants = ['NW', 'NE', 'SW', 'SE']

for i, ax in enumerate(axes.flat):
    for fail_satellite in values[f'f{quadrants[i]}']:
        g = sns.lineplot(x=values[f'f{quadrants[i]}'][fail_satellite][0], y=values[f'f{quadrants[i]}'][fail_satellite][1], ax=ax, color='lightgrey')
    for satellite in values[quadrants[i]]:
        g = sns.lineplot(x=values[quadrants[i]][satellite][0], y=values[quadrants[i]][satellite][1], ax=ax)
    g.set_ylabel('volts/volts')
    g.set_xlabel('reflector height (m)')

plt.tight_layout()
plt.show()

This is for L5:

In [ ]:
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, f=5)
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook: {station}', size=16)

quadrants = ['NW', 'NE', 'SW', 'SE']

for i, ax in enumerate(axes.flat):
    for fail_satellite in values[f'f{quadrants[i]}']:
        g = sns.lineplot(x=values[f'f{quadrants[i]}'][fail_satellite][0], y=values[f'f{quadrants[i]}'][fail_satellite][1], ax=ax, color='lightgrey')
    for satellite in values[quadrants[i]]:
        g = sns.lineplot(x=values[quadrants[i]][satellite][0], y=values[quadrants[i]][satellite][1], ax=ax)
    g.set_ylabel('volts/volts')
    g.set_xlabel('reflector height (m)')

plt.tight_layout()
plt.show()

### Analyze the Data
Now prepare to analyze the data using **gnssir**.  First you need to create a set of analysis instructions.
The default settings only need the station name, latitude, longitude, and ellipsoidal height.

In [ ]:
# Here we can see what parameters are available and what the defaults are:
#check_parameters.make_json?

In [ ]:
# we saved the lat, long, and height earlier in the notebook
run_gnssrefl.make_json(station, lat, long, height)

In [ ]:
# This is the json file that was created
json_file = f'{refl_code_loc}/input/lorg.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)

os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Next make some snr files for a time span of about eight months. Restrict the search to the UNAVCO archive to make the
code run faster (otherwise it will check three other archives as well).

Setting the parameter weekly=True will sample one rinex file per week over the time period requested - this is in the interest of saving time. It should only take a couple of minutes to load.

In [ ]:
#run_gnssrefl.rinex2snr?

In [ ]:
run_gnssrefl.rinex2snr(station, year=2019, doy=1, doy_end=233, archive='unavco')

Run **gnssir** for all the SNR files from **rinex2snr**.

In [ ]:
#run_gnssrefl.gnssir?

Unlike **quickLook**, **gnssir** plots the periodograms for each frequency with no quadrants.
The default does not send any plots to the screen. If you do want to see them, set -plt True. This is an example of running doy 1:

In [ ]:
doy = 1
run_gnssrefl.gnssir(station, year, doy=1, plt=True, screenstats=False)

Now we run this for all of the days we downloaded. Note* There will be a lot of 'Could not read the first SNR file'. This is due to the fact that we used the 'weekly' parameter to speed up notebook for this purpose - and it cannot find all of the days we are requesting. 

In [ ]:
doy = 1
doy_end=233
run_gnssrefl.gnssir(station, year, doy=doy, doy_end=doy_end, screenstats=False)

The **daily_avg** command will calculate the daily average reflector height from the daily output files. To minimize outliers in these daily averages, a median filter is set to 0.25 meters and the required minimum number of daily satellite tracks we will set to 50.

In [ ]:
#run_gnssrefl.daily_avg?

In [ ]:
run_gnssrefl.daily_avg(station, medfilter=.25, ReqTracks=50, plt2screen=False, txtfile='lorg-dailyavg.txt')

This shows the range of reflector heights each day, with each day represented by a different color.

There are also optional inputs for saving a text file of the daily averages. 
The plot is stored at $REFL_CODE/Files/lorg_RH.png 
This is not yet perfect - there are some outliers. 
But it is an excellent example of how easy it is to use the default gnssrefl code to estimate snow accumulation in Antarctica.

In this exercise L1, L2C, and L5 signals were used (i.e. only GPS data). The reflector heights give information 
about snow accumulation at lorg. Notice that when reflector heights are plotted, the y-axis is reversed so that the reflector height gets smaller as the snow layer increases and the distance between the antenna and snow surface decreases.  

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}_allRH.txt'
data = gnssrefl_helpers.read_allrh_file(filepath)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim(3.0, 1.8)
g.set_ylabel('Reflector Height (m)');

In [ ]:
plt.figure(figsize=(8,8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}-dailyavg.txt'
data = gnssrefl_helpers.read_allrh_file(filepath)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])

In [ ]:
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(2.7,2.1)
g.set_ylabel('Reflector Height (m)');